<a href="https://colab.research.google.com/drive/1d2alej4gnUKVcRgiWOVzzmMJzMp7IeIh?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### 🔄 Adaptive-RAG (Retrieval-Augmented Generation)

It is an advanced technique that improves upon traditional RAG systems by dynamically adapting the retrieval process based on the specific query and context. This approach aims to enhance the relevance and accuracy of retrieved information, leading to better-generated responses.

### Key features of Adaptive-RAG include:

1. Dynamic query reformulation
2. Iterative retrieval
3. Adaptive chunk selection
4. Relevance feedback

### Install required libraries

In [ ]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.3.19 \
     langchain-groq==0.2.4 \
     langchain-chroma==0.2.2 \
     langchain-community==0.3.18 \
     langchain-huggingface==0.1.2 \
     einops==0.8.1

### Import related libraries related to Langchain, HuggingfaceEmbedding

In [2]:
import numpy as np
from typing import List, Dict, Any
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain, RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [3]:
import getpass
import os

#### Provide a Groq API key. You can create one to access free open-source models at the following link.

[Groq API Creation Link](https://console.groq.com/keys)




In [4]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

··········


### Provide Huggingface API Key. You can create Huggingface API key at following lin

[Huggingface API Creation Link](https://huggingface.co/settings/tokens)




In [5]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


# The main flow of the system is:

1. Load documents into the vector store.
2. When a query is received, it retrieves relevant documents.
3. It then generates an answer based on these documents.
4. If the answer isn't satisfactory, it reformulates the query and tries again.
5. This process repeats until a satisfactory answer is found or the maximum number of iterations is reached.

In [6]:
class AdaptiveRAG:

    def __init__(self):
        """
        Initializes the AdaptiveRAG system.

        This method:
        - Sets up the language model (LLM) using Groq endpoint.
        - Initializes embeddings with the nomic.ai model.
        - Prepares placeholders for the vector store and retriever.
        """
        self.llm = ChatGroq(
            model="llama3-8b-8192",
            temperature=0.5
        )
        self.embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})
        self.vectorstore = None
        self.retriever = None

    def load_documents(self, documents: List[str]):
        """
        Processes and loads documents into the system.

        This method:
        - Splits documents into smaller chunks for improved processing.
        - Creates a Chroma vector store from these chunks.
        - Sets up a retriever to fetch relevant documents.
        """
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        texts = text_splitter.create_documents(documents)
        self.vectorstore = Chroma.from_documents(texts, self.embeddings)

        self.retriever = self.vectorstore.as_retriever(search_kwargs={"k": 4})

    def query_reformulation(self, query: str, context: str) -> str:
        """
        Creates a more targeted query from the original query and current context.

        This method:
        - Uses the LLM to generate a reformulated query based on the provided prompt.
        """
        prompt = f"""Given the original query and the current context, please reformulate the query to be more specific and targeted:

        Original query: {query}
        Current context: {context}

        Reformulated query:"""

        response = self.llm.invoke(prompt)
        return response.content

    def generate_answer(self, query: str, context: str) -> str:
        """
        Generates an answer to the query based on the given context.

        This method:
        - Uses the LLM to produce an answer using the provided prompt.
        """
        prompt = f"""Please answer the following query based on the given context:

        Query: {query}
        Context: {context}

        Answer:"""

        response = self.llm.invoke(prompt)
        return response.content

    def run(self, query: str, max_iterations: int = 3) -> Dict[str, Any]:
        """
        Runs the adaptive RAG process.

        This method iterates up to max_iterations times, performing the following steps each time:
        - Reformulates the query (after the first iteration).
        - Retrieves relevant documents.
        - Generates an answer.
        - Checks if the answer is satisfactory.

        It keeps track of all iterations and returns the final answer along with iteration details.
        """
        original_query = query
        context = ""
        iteration_results = []

        for i in range(max_iterations):
            if i > 0:
                query = self.query_reformulation(original_query, context)

            docs = self.retriever.get_relevant_documents(query)
            context = " ".join([doc.page_content for doc in docs])

            answer = self.generate_answer(query, context)

            iteration_results.append({
                "iteration": i + 1,
                "query": query,
                "answer": answer,
                "source_documents": docs
            })

            if self.is_answer_satisfactory(answer):
                break

        return {
            "final_answer": answer,
            "iterations": iteration_results
        }

    def is_answer_satisfactory(self, answer: str) -> bool:
        """
        Checks if an answer is satisfactory.

        This method:
        - Evaluates whether the answer is longer than 100 characters in this simple implementation.
        """
        return len(answer) > 100  # Simple length-based heuristic

# This example demonstrates how to use the AdaptiveRAG system:

1. Set it up with an API key
2. Load some documents
3. Ask a question
4. Get and display the results, including the process the system went through to arrive at the final answer

It's a simple demonstration, but it shows the core functionality of the adaptive RAG system, including its ability to potentially reformulate queries and generate answers over multiple iterations if needed.

In [ ]:
# Usage Example
adaptive_rag = AdaptiveRAG()

documents = [
    "Paris is the capital of France.",
    "London is the capital of the United Kingdom.",
    "Berlin is the capital of Germany."
]
adaptive_rag.load_documents(documents)

query = "What is the capital of France?"
result = adaptive_rag.run(query)

print(f"Final answer: {result['final_answer']}")
print(f"Number of iterations: {len(result['iterations'])}")
for i, iteration in enumerate(result['iterations']):
    print(f"\nIteration {i + 1}:")
    print(f"Query: {iteration['query']}")
    print(f"Answer: {iteration['answer']}")